In [38]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [39]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
⌃ [6e4b80f9] BenchmarkTools v1.5.0
⌅ [9136182c] ITensors v0.6.16
  [682c06a0] JSON v0.21.4
⌃ [33e6dc65] MKL v0.7.0
⌃ [91a5bcdd] Plots v1.40.5
⌃ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.10
⌃ [a759f4b9] TimerOutputs v0.5.24
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [40]:
# benchmark adder_MPO
using ITensors
include("CT/src/test_adder_MPO.jl")
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random
L = 4
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim)
# println(initial_state)

[1, 2, 3, 4]
[1, 4, 2, 3]
[1, 3, 4, 2]


MPS
[1] ((dim=2|id=152|"Qubit,Site,n=1"), (dim=8|id=305|"Link,l=1"))
[2] ((dim=8|id=305|"Link,l=1"), (dim=2|id=492|"Qubit,Site,n=2"), (dim=4|id=459|"Link,l=2"))
[3] ((dim=4|id=459|"Link,l=2"), (dim=2|id=728|"Qubit,Site,n=3"), (dim=2|id=606|"Link,l=3"))
[4] ((dim=2|id=606|"Link,l=3"), (dim=2|id=180|"Qubit,Site,n=4"))


In [46]:
# debug 
# haining
using .CT: add1, power_mpo
add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# tao
# Convert fractions to binary shifts
shift_1_6_bits, shift_1_6_amount = fraction_to_binary_shift(1, 6, L)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
println("shift_1_6_bits: ", shift_1_6_bits)
println("shift_1_3_bits: ", shift_1_3_bits)
# println("shift_1_6_amount: ", shift_1_6_amount)
# println("shift_1_3_amount: ", shift_1_3_amount)
# Create binary addition MPOs
# add_mpo_1_6 = create_binary_addition_mpo(shift_1_6_bits, qubit_site, phy_ram, phy_list)
add_mpo_1_3 = create_binary_addition_mpo(shift_1_3_bits, qubit_site, phy_ram, phy_list, ram_phy)

basis_state = productMPS(qubit_site, [1,1,2,1])
# println(basis_state)
# println(add_mpo_1_3)
result = basis_state * add_mpo_1_3
# println("~"^100)
println("result: ", vec(array(contract(result))))

# result_vec = vec(array(contract(result)))
# print(vector(result_vec))
# println("~"^100)
# n = mps_to_computational_basis(result)
# binary_str = string(n, base=2, pad=L)  # L is the number of qubits
# println("State: |$binary_str⟩")
# For your specific MPS structure
# Contract carry_phy_2 (tensors 1 and 3)


shift_1_6_bits: [0, 0, 1, 1]
shift_1_3_bits: [0, 1, 0, 1]
result: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [42]:
using ITensors

# Create your MPS
N = 4
sites = siteinds("S=1/2", N)
ψ = randomMPS(sites)

# Contract the MPS and convert to array
ψ_tensor = contract(ψ)
ψ_vector = array(ψ_tensor)  # Note: array() not vector()
ψ_flat = vec(ψ_vector)      # Then flatten with vec()

16-element Vector{Float64}:
 -0.01182084222318313
 -0.001662546025718169
 -0.007809907233068531
 -0.001098426828343982
 -0.7940980587997447
 -0.11168616810558502
 -0.5246523095471761
 -0.07378988701928833
  0.0033907506271505933
  0.00047689317503237893
  0.0022402335932188867
  0.0003150783494751238
  0.22778313423501897
  0.03203662965785548
  0.1504939423135328
  0.02116626725609475

In [43]:
# benchmark adder_MPO
using ITensors
include("CT/src/test_adder_MPO.jl")
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random
L = 4
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim)
println(phy_ram[phy_list[4]])


[1, 2, 3, 4]
[1, 3, 4, 2]
2


In [44]:
# benchmark adder_MPO
using ITensors
include("CT/src/test_adder_MPO.jl")
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random
L = 4
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim)
println(phy_ram[phy_list[4]])


[1, 2, 3, 4]
[1, 3, 4, 2]
2


In [45]:
# benchmark adder_MPO
using ITensors
include("CT/src/test_adder_MPO.jl")
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random
L = 4
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim)
println(phy_ram[phy_list[4]])


[1, 2, 3, 4]
[1, 3, 4, 2]
2
